In [55]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import numpy as np
# from torchinfo import summary
from sklearn.model_selection import train_test_split

In [56]:
# ładujemy tekst
with open('pantadeusz.txt', 'r', encoding='utf-8') as f:
    text = f.read().lower() # konwersja na małe litery

# tokenizacja
tokens = list(text)

vocab = sorted(set(tokens))
vocab_size = len(vocab)

# stworzenie słownika token -> indeks
token_to_index = {token: index for index, token in enumerate(vocab)}
index_to_token = {index: token for token, index in token_to_index.items()}

# konwersja tokenów na indeksy
indexed_tokens = [token_to_index[c] for c in tokens]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [58]:
class CharDataset(Dataset):
    def __init__(self, sequence, seq_length):
        self.sequence = sequence
        self.seq_length = seq_length
    
    def __len__(self):
        return len(self.sequence) - self.seq_length

    def __getitem__(self, index):
        return (torch.tensor(self.sequence[index:index+self.seq_length]),
                torch.tensor(self.sequence[index+1:index+self.seq_length+1]))

seq_length = 100

train_indices, test_indices = train_test_split(list(range(len(indexed_tokens))), test_size=0.1, random_state=42, shuffle=False)


# Tworzenie oddzielnych zestawów danych dla treningu i testu
train_dataset = CharDataset([indexed_tokens[i] for i in train_indices], seq_length)
test_dataset = CharDataset([indexed_tokens[i] for i in test_indices], seq_length)


train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class CharLSTM(pl.LightningModule):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers):
        super(CharLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.fc(lstm_out)
        return out
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss()(y_hat.transpose(1, 2), y)
        accuracy = (torch.argmax(y_hat, dim=2) == y).float().mean()
        self.log('loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('accuracy', accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss()(y_hat.transpose(1, 2), y)
        accuracy = (torch.argmax(y_hat, dim=2) == y).float().mean()
        self.log('test_loss', loss)
        self.log('test_accuracy', accuracy)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

# Initialize model
model = CharLSTM(vocab_size, embed_dim=64, hidden_dim=512, num_layers=2)  # experiment with these parameters
model = model.to(device)


In [ ]:
print(model)
summary(model.cuda(), (seq_length,))

In [ ]:
# PyTorch Lightning Trainer setup
trainer = pl.Trainer(max_epochs=100, callbacks=[EarlyStopping(monitor='loss')])  # experiment with max epochs
trainer.fit(model, train_dataloader)
trainer.test(model, test_dataloader)


In [59]:

def generate_text_mod(initial_str, model, length, temperature=1.0):
    model.eval()
    idxs = [token_to_index[c] for c in initial_str]
    input_seq = torch.tensor(idxs).unsqueeze(0)
    # input_seq = input_seq.to(device)

    generated_sequence = list(idxs)

    for _ in range(length):
        with torch.no_grad():
            output = model(input_seq)
            distribution = torch.softmax(output[0, -1]/temperature, dim=0).detach().numpy()
            next_char_idx = np.random.choice(len(distribution), p=distribution)

            generated_sequence.append(next_char_idx)

            input_seq = torch.tensor([generated_sequence])
    
    return ''.join(index_to_token[idx] for idx in generated_sequence)

# Generate some text
# print(generate_text_mod("pan tadeusz poszedl", model, 200))


In [ ]:
torch.save(model.state_dict(), 'model_seq20_state_dict.pth')
torch.save(model, 'model_seq20.pth')

In [ ]:
def calculate_character_level_accuracy(model, dataloader):
    model.eval()  # Przełącz model w tryb ewaluacji
    total_correct = 0
    total_characters = 0

    with torch.no_grad():  # Wyłącz obliczenia gradientów
        for batch in dataloader:
            sequences, targets = batch
            outputs = model(sequences)  # Wygeneruj przewidywania modelu
            _, predicted = torch.max(outputs, dim=2)  # Znajdź indeksy największych wartości

            total_correct += (predicted == targets).sum().item()  # Sumuj poprawne przewidywania
            total_characters += targets.numel()  # Zwiększ całkowitą liczbę znaków

    accuracy = total_correct / total_characters  # Oblicz dokładność
    return accuracy

accuracy = calculate_character_level_accuracy(model, test_dataloader)
print(f"Character-Level Accuracy: {accuracy:.2f}")

In [60]:
model = torch.load('model_seq100.pth')

print("Sequence length:", seq_length)

training_text = """Panno święta, co Jasnej bronisz Częstochowy
I w Ostrej świecisz Bramie! Ty, co gród zamkowy
Nowogródzki ochraniasz z jego wiernym ludem!
Jak mnie dziecko do zdrowia powróciłaś cudem"""

training_text = training_text.lower()
training_text = training_text[:seq_length]

print("Input text (from training dataset):")
print(training_text)
print("\nGenerated text:")
print(generate_text_mod(training_text, model, 200)[seq_length:])

test_text = """Tak za dni moich przy wiejskiej zabawie,
Czytano nieraz pod lipą na trawie
Pieśń o Justynie, powieść o Wiesławie;
A przy stoliku drewnianym pan włodarz"""

test_text = test_text.lower()
test_text = test_text[:seq_length]

print("\n\nInput text (from test dataset):")
print(test_text)
print("\nGenerated text:")
print(generate_text_mod(test_text, model, 200)[seq_length:])

outer_text = """Wsiąść do pociągu byle jakiego,
Nie dbać o bagaż, nie dbać o bilet,
Ściskając w ręku kamyk zielony,
Patrzeć jak wszystko zostaje w tyle"""

outer_text = outer_text.lower()
outer_text = outer_text[:seq_length]

print("\n\nInput text (from outside of dataset):")
print(outer_text)
print("\nGenerated text:")
print(generate_text_mod(outer_text, model, 200)[seq_length:])



Sequence length: 100
Input text (from training dataset):
panno święta, co jasnej bronisz częstochowy
i w ostrej świecisz bramie! ty, co gród zamkowy
nowogród

Generated text:
 w drabinach; dziś życie zawzos zestawicza,
bez w istoc, między zakładem nikita
jako środkowicz, aż wygrał się zając skrzywdziało».
dziś takim ujedwo serca list kąpi,
idąc sięm serca wznorze w bliska 


Input text (from test dataset):
tak za dni moich przy wiejskiej zabawie,
czytano nieraz pod lipą na trawie
pieśń o justynie, powieść

Generated text:
 wmiesz… wziął rad na kończyma:
bo przypadnęło: łajć głowę pewnie mu przebaczeni
szak wzruszony, trzęs dosyć dał każe,
swych miny i tak i gałęzistymi kurki
zwierzyła zioła zgodzili, znowu wynarły
jako


Input text (from outside of dataset):
wsiąść do pociągu byle jakiego,
nie dbać o bagaż, nie dbać o bilet,
ściskając w ręku kamyk zielony,


Generated text:
i biała rzewa lepszy padł na bauje, był w *rzeczym schwyta,
które niknął woje i służy i ostyga;
a w soplicowie ni